# Controle de LQ da Planta de Inércia
## Lucas Hattori Costa - 10335847

### Análise do sistema
Dá-se o nome de planta de inércia aos sistemas cuja única resistência ao movimento é dada pela massa em translação ou pelos seus momentos de inércia, de forma que as equações de movimento se tornam: $m \ddot{x}=f_{x} x t$ ou $J_{x} \dot{w}_{x}=M_{G, x}^{e x t}(x, y, z)$. Logo, podemos entender as forças externas como parâmetros de controle, de forma que $ u = \ddot{x} = \frac{f_{ext}}{m}$.

Logo, escrevemos o espaço de estados para o sistema:

$\left[\begin{array}{l}
\dot{x}_{1} \\
\dot{x}_{2}
\end{array}\right]=\left[\begin{array}{ll}
0 & 1 \\
0 & 0
\end{array}\right]\left[\begin{array}{l}
x_{1} \\
x_{2}
\end{array}\right]+\left[\begin{array}{l}
0 \\
1
\end{array}\right] u$

Resolvendo a equação de Riccati, podemos escrever as matrizes P, Q e R, e, em seguida, calcular o ganho K:

$P=\left[\begin{array}{ll}
1
\end{array}\right] \quad Q=\left[\begin{array}{ll}
1 & 0 \\
0 & \mu
\end{array}\right] \quad R=\left[\begin{array}{cc}
\sqrt{\mu+2} & 1 \\
1 & \sqrt{\mu+2}
\end{array}\right]$

$K=P^{-1} B^{T} R = \left[\begin{array}{ll}
1 & \sqrt{\mu+2}
\end{array}\right]$

Logo, com auxílio de um algoritmo em Python, podemos calcular a matriz K para as variações de valor de $\mu$ entre os valores 0,1; 0,5; 1,0; 5,0 e 10,0.

In [108]:
# importando as bibliotecas utilizadas
import numpy as np
import scipy as scipy
import matplotlib.pyplot as plt

import control as ctr
from control.matlab import *

In [79]:
mu_s = [0.1, 0.5, 1.0, 5.0, 10.0]
A = np.array([[0, 1],
    [0, 0]])
B = np.array([[0],[1]])
P = np.array([1])

for mu in mu_s:
    Q = np.array([[1, 0],
        [0, mu]])
    K,R,E = ctr.lqr(A, B, Q, P)
    K = K[0]
    print(f'Para o valor de mu={mu} temos a matriz K=[1, {K.item(1)}]')

Para o valor de mu=0.1 temos a matriz K=[1, 1.449137674618945]
Para o valor de mu=0.5 temos a matriz K=[1, 1.5811388300841898]
Para o valor de mu=1.0 temos a matriz K=[1, 1.7320508075688776]
Para o valor de mu=5.0 temos a matriz K=[1, 2.6457513110645916]
Para o valor de mu=10.0 temos a matriz K=[1, 3.464101615137754]


Para se simular as respostas do sistema, calcula-se de forma iterativa, utilizando a matriz de transição $\phi$ e a integral de convolução $\Gamma$, de tal forma que:
$\begin{aligned}
&\phi(\Delta t)=e^{A \Delta t}=\sum_{k=0}^{n-1} \frac{A^{k} \Delta t^{k}}{k !}\\
&\Gamma(\Delta t)=\int_{0}^{\Delta t} \phi(\Delta t-\tau) d \tau=\Delta t \sum_{k=0}^{n-1} \frac{A^{k} \Delta t^{k}}{(k+1) !}\\
&x_{i+1}=\phi x_{i}+\Gamma B u_{i}
\end{aligned}$

Adotando $n=20$ e $\Delta t =0,05s$ e $\mu=10$ como exemplo:

In [80]:
def _trans_conv(A, mu, delta_t = 0.05, n = 20):
    transicao = np.zeros((A.shape))
    convolucao = np.zeros((A.shape))
    delta_t = 0.05

    for n in range(20):
        transicao += np.multiply(A^n, delta_t**n)/ np.math.factorial(n)
        convolucao += np.multiply(A^n, delta_t**n)/ np.math.factorial(n+1)

    convolucao = convolucao * delta_t
    
    print(f'A matriz de transicao calculada para mu={mu} é: \n{transicao}\nA integral de convolução é:\n{convolucao}')

_trans_conv(A, mu)

A matriz de transicao calculada para mu=10.0 é: 
[[0.05256355 1.00379298]
 [0.05256355 0.05256355]]
A integral de convolução é:
[[0.00129246 0.05006303]
 [0.00129246 0.00129246]]


In [104]:
tempo = np.arange(0,10,delta_t)

In [105]:
def _resp(B, transicao, convolucao, K, tempo = tempo, x0 = [5, 0]):
    x = np.zeros((B.shape[0],len(tempo)))
    x[:,0] = x0
    for i in range(len(tempo) - 1):
        x[:,i+1] = np.dot(transicao, x[:,i]) + np.dot(np.dot(convolucao, B), np.dot(-K, x[:,i])).T
    return x

In [110]:
def _plot_resp(resp, tempo, mu):
    fig = plt.figure(figsize = (20,20))
    ax = plt.plot(resp[0], tempo)
    
    plt.show()

# _plot_resp(_resp(B, transicao, convolucao, K, tempo = tempo, x0 = [5, 0]), tempo, '10')

In [113]:
resp = _resp(B, transicao, convolucao, K, tempo = tempo, x0 = [5, 0])

In [115]:
plt.plot(resp, np)

array([5.00000000e+000, 1.25026044e-002, 2.12901047e-001, 1.13007793e-002,
       9.61011161e-003, 9.65905523e-004, 4.56847084e-004, 6.41167420e-005,
       2.26383215e-005, 3.86719574e-006, 1.15671019e-006, 2.22691176e-007,
       6.03718951e-008, 1.25080930e-008, 3.19575654e-009, 6.92678343e-010,
       1.70712168e-010, 3.80425636e-011, 9.17182005e-012, 2.07902184e-012,
       4.94549047e-013, 1.13279890e-013, 2.67259197e-014, 6.16113286e-015,
       1.44628702e-015, 3.34726115e-016, 7.83330041e-017, 1.81729888e-017,
       4.24484169e-018, 9.86244150e-019, 2.30100306e-019, 5.35097829e-020,
       1.24755054e-020, 2.90278587e-021, 6.76474408e-022, 1.57454769e-022,
       3.66839960e-023, 8.54026849e-024, 1.98939738e-024, 4.63203516e-025,
       1.07889327e-025, 2.51224936e-026, 5.85117110e-027, 1.36253563e-027,
       3.17330366e-028, 7.38974500e-029, 1.72100938e-029, 4.00782610e-030,
       9.33375805e-031, 2.17363670e-031, 5.06210186e-032, 1.17886543e-032,
       2.74540145e-033, 6